In [1]:
import pandas as pd

In [2]:
#load a sample set
df = pd.read_csv('test_Feb_2016-Mar_2018.csv')

In [3]:
def Date_filter(df, startYear, startMonth, endYear, endMonth):
    df['Year'] = df['DATE'].str.split('-', expand=True)[0]
    df['Month'] = df['DATE'].str.split('-', expand=True)[1]
    if startYear == endYear:
        df_mod = df[(df['Month'].astype(int) >= startMonth) & 
                (df['Month'].astype(int) <= endMonth) & 
                (df['Year'].astype(int) == startYear)]
        #df_mod.to_csv(file_name)
        return(df_mod)
    elif startYear <= endYear:
        df1 = df[(df['Year'].astype(int) >= startYear+1) & 
                 (df['Year'].astype(int) <= endYear-1)]
        df2 = df[(df['Month'].astype(int) >= startMonth) & 
                (df['Month'].astype(int) <= 12) & 
                (df['Year'].astype(int) == startYear)]
        df3 = df[(df['Month'].astype(int) >= 1) & 
                (df['Month'].astype(int) <= endMonth) & 
                (df['Year'].astype(int) == endYear)]
        df_mod = pd.concat([df1, df2, df3])
        return(df_mod)

In [4]:
AprOct_2016 = Date_filter(df,2016,4,2016,10)

In [5]:
Nov_2016 = Date_filter(df,2016,11,2016,11)

In [6]:
AprOct_2016 = AprOct_2016[AprOct_2016['AH1_RES_BUS_INDC'] == 'R']
AprOct_2016 = AprOct_2016[AprOct_2016['SUPP1_BUS_PANDER'] == 'N']
AprOct_2016_mod = AprOct_2016[AprOct_2016['EMAIL_OPTIN_IND'] == 'Y']

In [7]:
Nov_2016 = Nov_2016[Nov_2016['AH1_RES_BUS_INDC'] == 'R']
Nov_2016 = Nov_2016[Nov_2016['SUPP1_BUS_PANDER'] == 'N']
Nov_2016_mod = Nov_2016[Nov_2016['EMAIL_OPTIN_IND'] == 'Y']

# Label Function will start here:

In [8]:
def label_indiv(AprOct_file,Nov_file):
    tire_purchased = Nov_file[Nov_file['PROD_GROUP_CODE'] == 5]
    tire_purchased_id = tire_purchased['INDIV_ID'].drop_duplicates()
    target = pd.merge(AprOct_file,tire_purchased_id,'right',['INDIV_ID', 'INDIV_ID'])
    non_target = AprOct_file.append(target).drop_duplicates(keep = False)
    target['label'] = 1
    non_target['label'] = 0
    AprOct_labeled = pd.concat([target,non_target])
    #AprOct_labeled.to_csv(file_name,index=False)
    return(AprOct_labeled)

# Another Label Function will start here:

#### Due to the first one may remove some existed duplicates, the second one would run lower, but nothing would be deleted.

In [9]:
def label_act(AprOct_file,Nov_file):
    tire_purchased = Nov_file[Nov_file['PROD_GROUP_CODE'] == 5]
    id_list = tire_purchased['INDIV_ID'].drop_duplicates().tolist()
    label_list = []
    for i in range(len(AprOct_file)):
        if AprOct_file.iloc[i]['INDIV_ID'] in id_list:
            label_list.append(1)
        else:
            label_list.append(0)
    AprOct_file['label'] = label_list
    AprOct_labeled = AprOct_file
    #AprOct_labeled.to_csv('testabc.csv',index=False)
    return(AprOct_labeled)